In [1]:
import sys
sys.path.append('/mnt/projects/sne_students/M.Thesis_Scott/git_repos/GNN_MBTR_MD/graphNN_tools/')
import gnn_tools as gnn

import torch
from torch_geometric.data import InMemoryDataset
from torch_geometric.data import Dataset


In [2]:
import numpy as np
import pandas as pd
import itertools
import math
import pickle
from sklearn.preprocessing import StandardScaler
from torch_geometric.nn import NNConv, GATConv, Set2Set
from torch_geometric.data import Data
from torch_geometric.data import InMemoryDataset
from torch_geometric.nn import (graclus, max_pool, max_pool_x,
                                global_mean_pool)
from torch.nn import Sequential as Seq, Linear, ReLU, LeakyReLU, GRU
from torch_geometric.nn import MessagePassing
from torch_geometric.utils import remove_self_loops, add_self_loops,add_remaining_self_loops
from torch.nn import Parameter
from torch_geometric.nn.conv import MessagePassing
from inits import reset, uniform

from torch_geometric.nn import GraphConv, TopKPooling, GatedGraphConv
from torch_geometric.nn import global_mean_pool as gap, global_max_pool as gmp
import torch.nn.functional as F
import torch.nn as nn
from torch.nn import BatchNorm1d as BN
from torch.nn import LayerNorm as LN
from sklearn.metrics import mean_absolute_error, r2_score
from torch_geometric.nn import global_mean_pool as gap, global_max_pool as gmp

from torch_geometric.data import DataLoader
from pathlib import Path


# The dataset class

In [3]:
class transferIntegral(InMemoryDataset):
    def __init__(self, root, transform=None, pre_transform=None):
        super(transferIntegral, self).__init__(root, transform, pre_transform)
        self.data, self.slices = torch.load(self.processed_paths[0])
    @property
    def raw_file_names(self):
        return []
    @property
    def processed_file_names(self):
        return ['processed_data.dataset']
    def download(self):
        pass
    def process(self):
        pass


# The model

In [4]:
class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        '''
        nn (torch.nn.Module) – A neural network ℎ𝚯 that maps edge features
        edge_attr of shape [-1, num_edge_features] to
        shape [-1, in_channels * out_channels], e.g.,
        defined by torch.nn.Sequential.
        '''
        # number of neurons in dense nn
        p1 = 28
        p2 = 28
        # get features from dataset, but hardcode this for now
#       nbr_node_features = dataset[0].x.size()[1]
#       nbr_edge_features = dataset[0].edge_attr.size()[1]
        nbr_node_features = 2
        nbr_edge_features = 1

        totNbrFeatures = 0
        totNbrFeatures += p2*2

        self.lin0 = torch.nn.Linear(nbr_node_features, p2, bias = False)
        self.BN0 = BN(round(p2))

        nn = Seq(Linear(nbr_edge_features, p1, bias = False), BN(p1), LeakyReLU(), Linear(p1, p2 * p2, bias = False), BN(p2 * p2))

        self.conv = NNConv(p2, p2,nn, aggr='mean')

        self.set2set = Set2Set(p2, processing_steps=3)
        self.gru = GRU(p2, p2)

        print("totNbrFeatures", totNbrFeatures)
        self.lin1 = torch.nn.Linear(totNbrFeatures, round(totNbrFeatures/2))
        self.lin2 = torch.nn.Linear(round(totNbrFeatures/2), round(totNbrFeatures/4))
        self.lin_final = torch.nn.Linear(round(totNbrFeatures/4), 1)

    def forward(self, data):
        y = None
        x, edge_index, edge_attr, batch = data.x, data.edge_index, data.edge_attr, data.batch
        out = F.leaky_relu(self.BN0(self.lin0(x)))
        h = out.unsqueeze(0)

        for i in range(3):
            m = F.leaky_relu(self.conv(out, edge_index, edge_attr))
            out, h = self.gru(m.unsqueeze(0), h)
            out = out.squeeze(0)

        y_gnn = self.set2set(out, batch)
        y = y_gnn

        y = F.dropout(y, p = 0.5, training=self.training)
        y = F.leaky_relu(self.lin1(y))
        y = F.leaky_relu(self.lin2(y))
        y = self.lin_final(y)
        y = y.squeeze(-1)
        return y


# Make the data class

In [11]:
dataset = transferIntegral(root='data')
print(len(dataset))
print("Node features: {}".format(dataset[0].x))
print("Edge attributes: {}".format(dataset[0].edge_attr))
print("Tranfer integral: {}".format(dataset[0].y))

48818
Node features: tensor([[1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 9.],
        [1., 6.],
        [1., 6.],
        [1., 8.],
        [1., 6.],
        [1., 6.],
        [1., 1.],
        [1., 6.],
        [1., 1.],
        [1., 6.],
        [1., 1.],
        [1., 1.],
        [1., 6.],
        [1., 6.],
        [1., 1.],
        [1., 6.],
        [1., 1.],
        [1., 6.],
        [1., 1.],
        [1., 6.],
        [1., 1.],
        [1., 6.],
        [1., 1.],
        [1., 6.],
        [1., 1.],
        [1., 1.],
        [1., 6.],
        [1., 9.],
        [1., 9.],
        [1., 6.],
        [1., 1.],
        [1., 1.],
        [1., 6.],
        [1., 1.],
        [1., 6.],
        [1., 1.],
        [1., 6.],
        [1., 1.],
        [1., 6.],
        [1., 1.],
        [1., 6.],
        [1., 9.],
        [1., 9.],
        [1., 9.],
        [2., 8.],
        [2., 8.]])
Edge attributes: tensor([[-0.5907],
        [-0.5907],
        [-0.4019],
        ...,
        [